# VIX - Volatility Index.
It measures the market’s expected 30-day volatility from S&P 500 options.
- Low VIX → calm markets
- High VIX → fear / uncertainty
It’s often called the “fear index.”

#### How VIX impacts energy stocks
- Direct impact: None.
- Indirect impact: Through risk appetite and capital flows.

#### When VIX rises:
- Investors reduce risk
- Cyclical sectors (like energy) often sell off
- Oil demand expectations may fall
- Energy stocks typically drop

#### When VIX falls:
- Risk-on environment
- Cyclicals outperform
- Energy stocks often benefit

In [7]:
# ===================== CONFIG: VIX =====================
from pathlib import Path
import pandas as pd
from pandas_datareader import data as web

START_DATE = "2010-01-01"   # adjust if you want a later start
VIX_SERIES = "VIXCLS"       # CBOE Volatility Index (close) from FRED

BASE_DIR   = Path(r"D:\MS_Data_Science_Thesis\Data_Extraction")
RAW_DIR    = BASE_DIR / "Downloaded_datasets"
CLEAN_DIR  = BASE_DIR / "Raw_Data_Folder"

RAW_VIX_CSV   = RAW_DIR   / "vix_fred_raw.csv"
CLEAN_VIX_CSV = CLEAN_DIR / "vix_daily.csv"
# ======================================================


# ---------- ensure folders exist (safe if already created) ----------
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)


def fetch_vix_from_fred(series_id: str, start_date: str) -> pd.DataFrame:
    """
    Fetch VIX index from FRED using pandas_datareader.

    Returns a DataFrame with a DatetimeIndex and a single column `series_id`.
    """
    print(f"[vix] Fetching FRED series {series_id!r} from {start_date} onward...")
    df = web.DataReader(series_id, "fred", start=start_date)
    df = df.sort_index()
    return df


def clean_vix_df(df_raw: pd.DataFrame, start_date: str) -> pd.DataFrame:
    """
    Clean raw FRED VIX series:
      - filter from start_date
      - drop missing values
      - reset index and rename to tidy format
    """
    df = df_raw.copy()

    # Filter by start date (safety)
    df = df[df.index >= pd.to_datetime(start_date)]

    # Drop missing values
    df = df.dropna(how="any")

    # Reset index so date is a column
    df = df.reset_index()

    # Expect 2 columns: date + value
    if df.shape[1] != 2:
        print("[vix] WARNING: unexpected column structure in raw data:", df.columns.tolist())

    date_col = df.columns[0]
    value_col = df.columns[1]

    df = df.rename(
        columns={
            date_col: "date",
            value_col: "vix_index",
        }
    )

    # Enforce dtypes
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])

    df["vix_index"] = pd.to_numeric(df["vix_index"], errors="coerce")
    df = df.dropna(subset=["vix_index"])

    # Final tidy ordering
    df = df.sort_values("date")[["date", "vix_index"]]

    return df


# ---------- run pipeline ----------
try:
    # 1. Fetch from FRED (raw)
    vix_raw = fetch_vix_from_fred(VIX_SERIES, START_DATE)

    # 2. Save raw as reference
    vix_raw.to_csv(RAW_VIX_CSV)
    print(f"[vix] Saved raw FRED VIX series to {RAW_VIX_CSV}")

    # 3. Clean and standardize
    vix_clean = clean_vix_df(vix_raw, START_DATE)

    # 4. Save clean daily CSV
    vix_clean.to_csv(CLEAN_VIX_CSV, index=False)
    print(f"[vix] Saved clean daily VIX index to {CLEAN_VIX_CSV}")

    # 5. Quick QA printout
    if not vix_clean.empty:
        dmin = vix_clean["date"].min().date()
        dmax = vix_clean["date"].max().date()
        print(f"[vix] rows={len(vix_clean)}  range={dmin} → {dmax}")
    else:
        print("[vix] WARNING: clean dataset is empty after filtering/cleaning.")

except Exception as e:
    print("[vix] ERROR while fetching or processing VIX data:", repr(e))


[vix] Fetching FRED series 'VIXCLS' from 2010-01-01 onward...
[vix] Saved raw FRED VIX series to D:\MS_Data_Science_Thesis\Data_Extraction\Downloaded_datasets\vix_fred_raw.csv
[vix] Saved clean daily VIX index to D:\MS_Data_Science_Thesis\Data_Extraction\Raw_Data_Folder\vix_daily.csv
[vix] rows=4079  range=2010-01-04 → 2026-02-10
